In [ ]:
from google.auth import default
credentials, project = default()

In [ ]:
%load_ext autoreload
%autoreload 2
import os
import sys
from pathlib import Path
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from dotenv import load_dotenv
load_dotenv()

from src.utils import (load_json, get_first_n_items, save_json, save_text_file, load_text_file, clean_filename, read_from_gcs, upload_to_gcs)  # noqa: E402
from src.config_loader import config
config.TARGET_LANGUAGE_NAME

# Phrase Audio

## Storage overview

Save MP3 files of audio with the filenames of the phrase keys based on clean_filename

multimedia/audio/phrases/french/normal/this_is_a_phrase.mp3
multimedia/audio/phrases/french/slow/this_is_a_phrase.mp3

This allows us to quickly retrieve a phrase based on the bucket name and the phrase key, as well as modify individual phrases for later correction

In [ ]:
language_name = config.TARGET_LANGUAGE_NAME.lower()
print(language_name)
config.get_voice_models()

In [ ]:
# get translations

phrase_translations = read_from_gcs(bucket_name=config.GCS_PRIVATE_BUCKET, file_path=f"collections/LM1000/translations/{config.TARGET_LANGUAGE_NAME.lower()}.json")

In [ ]:
test_dict = get_first_n_items(phrase_translations, 3)

In [ ]:
test_dict

### Test a sample

In [ ]:
from src.audio_generation import generate_translated_phrase_audio

example_pair = test_dict['he_might_offer_to_help_with_the_work']
phrase_tuple = example_pair['english'], example_pair[language_name]

In [ ]:
audio = generate_translated_phrase_audio([phrase_tuple])

In [ ]:
audio[0][1]

### Upload phrases to GCS

In [ ]:
# Just directly await the coroutine in Jupyter
from src.audio_generation import upload_phrases_audio_to_gcs
result = upload_phrases_audio_to_gcs(phrase_translations)

In [ ]:
list(result.keys())[-1]